# **Fantasy Model Application and Database**

---

This application aims to keep a historical record of any fantasy football league and provide some visualizations for league data.
<br><br>
***HOW TO RUN THIS APPLICATION:***
- First, run the **Build** section. This sets up global functions and the application entities.
- Then, run the **Create** section. This creates the application and adds data to the database.
- Finally, run any of the cells in the **Run** section in any order.

## Build

---

You can collapse this section and run all cells to set up the application and database.

### Imports

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import decimal
from IPython.display import display

### Classes

#### Application

In [2]:
class Application():
  def __init__(self):
    self.db = Database()
    self.visualizer = Visualizer()

  def displayLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df)
  
  def displayLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df, self.db.manager_record_df)

  def displayActiveLeagueManagers(self):
    self.visualizer.createLeagueManagersDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0])
  
  def displayActiveLeagueManagers2(self):
    self.visualizer.createLeagueManagersDisplay2(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df)

  def displayManagerRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df, self.db.manager_record_df, sort_key)

  def displayActiveManagerRecords(self, sort_key):
    self.visualizer.createManagerRecordDisplay(self.db.manager_df.loc[self.db.manager_df['active'] > 0], self.db.manager_record_df, sort_key)

  def displayRecordsForManager(self, manager):
    # display(self.db.manager_record_df.loc[self.db.manager_record_df['abbreviation'] == manager].style.hide_index())
    self.visualizer.createRecordsForManagerDisplay(self.db.manager_df, self.db.manager_record_df, manager)

  def displayTotalChampionshipsByManager(self):
    self.visualizer.createTotalChampionshipsDisplay(pd.merge(self.db.manager_df, self.db.top_three_finish_df, on='abbreviation'))

  def displayTopThreeFinishes(self):
    self.visualizer.createTopThreeFinishesDisplay(self.db.manager_df, self.db.top_three_finish_df)

#### Database

In [3]:
class Database():
  def __init__(self):
    self.manager_df = ManagerHelper.createManagerDF()
    self.manager_record_df = RecordHelper.createManagerRecordDF()
    self.game_df = GameHelper.createGameDF()
    self.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()
    self.playoff_game_df= PlayoffHelper().createPlayoffGameDF()

  def addManagers(self, managers):
    self.manager_df = ManagerHelper.addManagers(self.manager_df, managers)

  def addActiveYears(self):
    self.manager_df = ManagerHelper.addActiveYears(self.manager_df, self.manager_record_df)

  def addGamesForYearAndWeek(self, year, week, games):
    self.game_df = GameHelper.addGamesForYearAndWeek(self.game_df, year, week, games)

  def addLegacyRecordsForYear(self, year, records):
    self.manager_record_df = RecordHelper.addLegacyRecordsForYear(self.manager_record_df, year, records)

  def addPlayoffGamesForYearAndRound(self, year, game_round, playoff_games):
    self.playoff_game_df = PlayoffHelper.addPlayoffGamesForYearAndRound(self.playoff_game_df, year, game_round, playoff_games)

  def addTopThreeFinishesForYear(self, year, finishes):
    self.top_three_finish_df = TopThreeFinishHelper.addTopThreeFinishesForYear(self.top_three_finish_df, year, finishes)

  def addRecordsFromGames(self):
    self.manager_record_df = RecordHelper.addRecordsFromGames(self.game_df, self.manager_record_df)

##### Manager Helper

In [4]:
class ManagerHelper():
  # Create Manager DataFrame
  def createManagerDF():
    df = pd.DataFrame()
    df['name'] = []
    df['abbreviation'] = []
    df['active'] = []
    df = df.astype({
        'active': int })
    return df

  def addManager(name, abbreviation, active, manager_df):
    dict = {
        'name': name,
        'abbreviation': abbreviation,
        'active': active
    }
    manager_df = manager_df.append(dict, ignore_index = True)
    return manager_df

  def addManagers(manager_df, managers):
    for manager in managers:
      name = manager[0]
      abbreviation = manager[1]
      active = manager[2]
      manager_df = ManagerHelper.addManager(name, abbreviation, active, manager_df)
    return manager_df

  def addActiveYears(manager_df, manager_record_df):
    return_df = pd.DataFrame()
    return_df['abbreviation'] = []
    return_df['active_years'] = []
    manager_list = manager_df['abbreviation']
    for manager in manager_list:
      result_df = manager_record_df.loc[manager_record_df['abbreviation'] == manager]
      active_years = ""
      for index, row in result_df.iterrows():
        year = row['season_start_year']
        active_years += str(year) + ", "
      active_years = active_years[0:-2]
      dict = {
          'abbreviation': manager,
          'active_years': active_years
      }
      return_df = return_df.append(dict, ignore_index=True)
    df = pd.merge(manager_df, return_df, on='abbreviation')
    return df

##### Record Helper

In [5]:
class RecordHelper():
  # Create League Manager Records DataFrame
  def createManagerRecordDF():
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['wins'] = []
    df['losses'] = []
    df['ties'] = []
    df['pf'] = []
    df['pa'] = []
    df = df.astype({
        'season_start_year': int,
        'wins': int,
        'losses': int,
        'ties': int })
    return df

  def addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': 0
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'wins': wins,
        'losses': losses,
        'ties': ties
    }
    manager_record_df = manager_record_df.append(dict, ignore_index = True)
    return manager_record_df

  def addLegacyRecordsForYear(manager_record_df, season_start_year, records):
    for record in records:
      manager = record[0]
      wins = record[1]
      losses = record[2]
      if len(record) == 4:
        ties = record[3]
        manager_record_df = RecordHelper.addLegacyRecordWithTies(manager_record_df, manager, season_start_year, wins, losses, ties)
        continue
      manager_record_df = RecordHelper.addLegacyRecord(manager_record_df, manager, season_start_year, wins, losses)
    return manager_record_df

  def addRecordsFromGames(game_df, manager_record_df):
    calculated_df = pd.DataFrame()
    calculated_df['abbreviation'] = pd.Series(dtype='object')
    calculated_df['season_start_year'] = pd.Series(dtype=int)
    calculated_df['wins'] = pd.Series(dtype=int)
    calculated_df['losses'] = pd.Series(dtype=int)
    calculated_df['ties'] = pd.Series(dtype=int)
    calculated_df['pf'] = pd.Series(dtype=float)
    calculated_df['pa'] = pd.Series(dtype=float)
    years_in_game_df = game_df['season_start_year'].unique()
    for year in years_in_game_df:
      year_game_df = game_df.loc[game_df['season_start_year'] == year]
      manager_list1 = np.array(year_game_df['manager1'].unique())
      manager_list2 = np.array(year_game_df['manager2'].unique())
      manager_list = np.concatenate((manager_list1, manager_list2))
      manager_list = np.unique(manager_list)
      for manager in manager_list:
        manager_games1 = year_game_df.loc[year_game_df['manager1'] == manager]
        manager_games2 = year_game_df.loc[year_game_df['manager2'] == manager]
        df = pd.concat([manager_games1, manager_games2])
        wins = 0
        losses = 0
        ties = 0
        pf = 0
        pa = 0
        for index, row in df.iterrows():
          manager1_score = row['manager1_score']
          manager2_score = row['manager2_score']
          if (row['manager1'] == manager):
            if (manager1_score > manager2_score):
              wins += 1
            elif (manager1_score < manager2_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager1_score
            pa += manager2_score
          else:
            if (manager2_score > manager1_score):
              wins += 1
            elif (manager2_score < manager1_score):
              losses += 1
            else:
              ties = RecordHelper.determine_tie(row, manager)
            pf += manager2_score
            pa += manager1_score
        dict = {
            'abbreviation': manager,
            'season_start_year': year,
            'wins': wins,
            'losses': losses,
            'ties': ties,
            'pf': pf,
            'pa': pa
        }
        calculated_df = calculated_df.append(dict, ignore_index = True)
    df = pd.concat([manager_record_df, calculated_df])
    return df

  def determine_tie(row, manager):
    return 0

##### Game Helper

In [6]:
class GameHelper():
  # Create Regular Season Games DataFrame
  def createGameDF():
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_score'] = []
    df['season_start_year'] = []
    df['week'] = []
    df = df.astype({
        'season_start_year': int, 
        'week': int })
    return df

  def addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week):
    dict = {
        'manager1': manager1, 
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_score': manager2_score,
        'season_start_year': season_start_year,
        'week': week
    }
    game_df = game_df.append(dict, ignore_index = True)
    return game_df

  def addGamesForYearAndWeek(game_df, year, week, games):
    for game in games:
      manager1 = game[0]
      manager1_score = game[1]
      manager2 = game[2]
      manager2_score = game[3]
      season_start_year = year
      week = week
      game_df = GameHelper.addGame(game_df, manager1, manager1_score, manager2, manager2_score, season_start_year, week)
    return game_df

##### Top Three Finish Helper

In [7]:
# TODO - make this top four

class TopThreeFinishHelper():
  # Create League Manager Top Three Finishes DataFrame
  def createTopThreeFinishesDF(self):
    df = pd.DataFrame()
    df['abbreviation'] = []
    df['season_start_year'] = []
    df['finishing_position'] = []
    df = df.astype({
        'season_start_year': int,
        'finishing_position': int })
    return df

  def addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position):
    dict = {
        'abbreviation': manager,
        'season_start_year': season_start_year,
        'finishing_position': finishing_position
    }
    top_three_finish_df = top_three_finish_df.append(dict, ignore_index=True)
    return top_three_finish_df

  def addTopThreeFinishesForYear(top_three_finish_df, season_start_year, finishes):
    for finish in finishes:
      manager = finish[0]
      finishing_position = finish[1]
      top_three_finish_df = TopThreeFinishHelper.addTopThreeFinish(top_three_finish_df, manager, season_start_year, finishing_position)
    return top_three_finish_df

##### Playoff Helper

In [8]:
class  PlayoffHelper():
  # Create Playoff Game DataFrame
  def createPlayoffGameDF(self):
    df = pd.DataFrame()
    df['manager1'] = []
    df['manager1_rank'] = []
    df['manager1_score'] = []
    df['manager2'] = []
    df['manager2_rank'] = []
    df['manager2_score'] = []
    df['round'] = []
    df['season_start_year'] = []
    df = df.astype({
        'season_start_year': int, 
        'round': int,
        'manager1_rank': int,
        'manager2_rank': int })
    return df

  def addPlayoffGamesForYearAndRound(playoff_game_df, year, round, playoff_games):
    for game in playoff_games:
      manager1 = game[0]
      manager1_rank = game[1]
      manager1_score = game[2]
      manager2 = game[3]
      manager2_rank = game[4]
      manager2_score = game[5]
      playoff_game_df = PlayoffHelper.addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year)
    return playoff_game_df

  def addPlayoffGame(playoff_game_df, manager1, manager1_rank, manager1_score, manager2, manager2_rank, manager2_score, round, year):
    dict = {
        'manager1': manager1, 
        'manager1_rank': manager1_rank,
        'manager1_score': manager1_score, 
        'manager2': manager2, 
        'manager2_rank': manager2_rank,
        'manager2_score': manager2_score,
        'round': round,
        'season_start_year': year
    }
    playoff_game_df = playoff_game_df.append(dict, ignore_index = True)
    return playoff_game_df

##### Sorting Helper

In [9]:
class sort_by():
  # Define Sort Keys
  name = 'name'
  wins = 'wins'
  losses = 'losses'
  ties = 'ties'
  win_pct = 'win_pct'

#### Visualizer

In [10]:
class Visualizer():
  def __init__(self):
    pd.set_option('max_colwidth', None)
    pd.set_option('colheader_justify', 'center')

  def createLeagueManagersDisplay(self, df):
    df = df.sort_values(by=['name'])
    df = df[['name', 'active_years']]
    df = df.rename(columns={
        'name':'Name', 
        'active_years': 'Active Years' })
    df = df.style.hide_index()
    display(df)

  def createLeagueManagersDisplay2(self, manager_df, manager_record_df):
    # select name and active years column sorted by name
    manager_df = manager_df[['name', 'active_years']]
    manager_df = manager_df.sort_values(by=['name'])

    # create empty display df and dict with name and years
    display_df = pd.DataFrame()
    display_df['name'] = []
    dict = {'name':''}
    years = manager_record_df['season_start_year'].unique()
    for year in years:
      display_df[year] = []
      dict[year] = ''

    # add green bar for active years
    managers_list = manager_df['name']
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      manager_active_years = manager_df.loc[manager_df['name'] == manager]['active_years']
      for year in years:
        if str(year) in str(manager_active_years):
          dict[year] = "🟩🟩🟩"
        else:
          dict[year] = ""
      display_df = display_df.append(dict, ignore_index=True)

    # clean up and display
    display_df = display_df.rename(columns={'name':'Name'})
    display_df = display_df.style.hide_index()
    display(display_df)

  def createManagerRecordDisplay(self, manager_df, manager_record_df, sort_key):
    # join manager_df and manager_record_df
    df = pd.merge(manager_df, manager_record_df, on='abbreviation')
    # get total wins, losses, and ties
    df = df.groupby('name', as_index=False).sum()
    # select name, wins, losses, and ties
    df = df[['name', 'wins', 'losses', 'ties']]

    # calculate win percentsge
    df['win_pct'] = pd.Series(dtype='int')
    for index, row in df.iterrows():
      wins = row['wins']
      losses = row ['losses']
      ties = row['ties']
      total_games = wins + losses + ties
      win_percent = wins / total_games
      c = decimal.Decimal(win_percent * 100)
      df.at[index, 'win_pct'] = float(round(c, 3))

    # sort by sort_key
    df = df.sort_values(by=[sort_key], ascending=False)

    # clean up and display
    df = df.rename(columns={
        'name':'Name', 
        'wins': 'Wins',
        'losses': 'Losses',
        'ties': 'Ties',
        'win_pct': 'Win Pct' })
    df = df.style
    df = df.format(precision=3)
    df = df.hide_index()
    display(df)

  def createTotalChampionshipsDisplay(self, df):
    df = df.loc[df['finishing_position'] == 1]
    df = df.groupby('name', as_index=False).sum()
    df = df.sort_values(by=['finishing_position'], ascending=False)
    df = df[['name', 'finishing_position']]
    df = df.rename(columns={'name':'Name','finishing_position': 'Championships'})
    df = df.style.hide_index()
    display(df)

  def createTopThreeFinishesDisplay(self, manager_df, top_three_finish_df):
    df = pd.merge(manager_df, top_three_finish_df, on='abbreviation')
    df2 = df[['name', 'season_start_year', 'finishing_position']]
    display_df = pd.DataFrame()
    display_df['name'] = []
    years_in_top_3_finishes = top_three_finish_df['season_start_year'].unique()
    dict = {'name':''}
    for year in years_in_top_3_finishes:
      display_df[year] = []
      dict[year] = ''
    dict['total'] = []
    managers_list = df2['name'].unique()
    for manager in managers_list:
      dict = {}
      dict['name'] = manager
      top_three_finishes = df2.loc[df2['name'] == manager][['season_start_year', 'finishing_position']]
      total_top_three = 0
      for year in years_in_top_3_finishes:
        if (top_three_finishes['season_start_year'].eq(year)).any():
          finishing_position = df[(df['season_start_year'] == year) & (df['name'] == manager)]
          finishing_position = finishing_position['finishing_position'].item()
          if (finishing_position == 3):
            dict[year] = "🥉"
          elif (finishing_position == 2):
            dict[year] = "🥈"
          else:
            dict[year] = "🥇"
          total_top_three += 1
        else:
          dict[year] = ""
        dict['total'] = total_top_three
      display_df = display_df.append(dict, ignore_index=True)
    display_df = display_df.astype({'total': int})
    display_df = display_df.sort_values(by=['total'], ascending=False)
    display_df = display_df.rename(columns={'name':'Name','total': 'Total'})
    display_df = display_df.style.hide_index()
    pd.set_option('colheader_justify', 'left')
    display(display_df)

  def createRecordsForManagerDisplay(self, manager_df, manager_record_df, manager):
    df = pd.merge(manager_df, manager_record_df, on='abbreviation')
    df = df.loc[df['name'] == manager]
    df = df[['name', 'season_start_year',	'wins',	'losses',	'ties',	'pf',	'pa']]
    df = df.fillna("-")
    df = df.rename(columns={
        'name':'Name',
        'season_start_year': 'Year',
        'wins': "Wins",
        'losses': 'Losses',
        'ties': 'Ties',
        'pf': 'PF',
        'pa': 'PA' })
    df = df.sort_values(by=['Year'])
    df = df.style
    df = df.set_properties(**{'text-align': 'center'})
    df = df.format(precision=2)
    df = df.hide_index()
    df.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df)

## Create

---

You can collapse this section and run all cells to create the application and add data to the database.

### Initialize

In [35]:
# Initilaize Application
app = Application()

### Add Managers

In [36]:
# Add Managers

# Reset Managers DataFrame
app.db.manager_df = ManagerHelper.createManagerDF()

# Create Mananger Abbreviations
CARR = 'CARR'
BERG = 'BERG'
BRKS = 'BRKS'
SHAR = 'SHAR'
FLAN = 'FLAN'
JOSH = 'JOSH'
ROBL = 'ROBL'
DVIS = 'DVIS'
GRIG = 'GRIG'
PETE = 'PETE'
KITW = 'KITW'
RANE = 'RANE'
SHAB = 'SHAB'
ALEX = 'ALEX'
PRUS = 'PRUS'
GABE = 'GABE'

# Add Manager Name, Abbreviation, and Active Status to DB
app.db.addManagers([
    ['Brendan Carr', CARR, 1],
    ['Jared Birnberg', BERG, 1],
    ['Brooks Langley', BRKS, 1],
    ['Cameron Sharer', SHAR, 1],
    ['Will Flanagan', FLAN, 1],
    ['Josh Parker', JOSH, 1],
    ['Rob Lewis', ROBL, 1],
    ['Davis Hodge', DVIS, 1],
    ['Drew Griggs', GRIG, 1],
    ['Peter Thropp', PETE, 1],
    ['Kit Williams', KITW, 1],
    ['Rhane Jones', RANE, 1],
    ['Utshab Chakraborty', SHAB, 1],
    ['Alex Nilsson', ALEX, 1],
    ['Max Pruss', PRUS, 1],
    ['Gabe Sugerman', GABE, 1]
])

### Add Records

In [37]:
# Add Legacy Records

# Reset Records DB
app.db.manager_record_df = RecordHelper.createManagerRecordDF()

### Add Games

In [38]:
# Reset Games DF
app.db.game_df = GameHelper.createGameDF()

#### 2022

In [39]:
# Add Regular Season Games for a Year

# Week 1
app.db.addGamesForYearAndWeek(2022, 1, [
    [CARR, 93.65, KITW, 121.75],
    [GRIG, 132.85, FLAN, 149.7],
    [PETE, 98.3, JOSH, 98.1],
    [DVIS, 112.65, ROBL, 121.65],
    [SHAR, 100.95, BRKS, 142.15],
    [ALEX, 88.9, BERG, 67.9],
    [SHAB, 119.2, GABE, 117.15],
    [RANE, 141.3, PRUS, 123.8]
])

# Week 2
app.db.addGamesForYearAndWeek(2022, 2, [
    [CARR, 95.8, PRUS, 149.05],
    [PETE, 136.75, GRIG, 141.65],
    [FLAN, 97.05, DVIS, 138.1],
    [JOSH, 106.9, SHAR, 116.25],
    [ROBL, 107.55, BRKS, 168.2],
    [SHAB, 100.1, ALEX, 115.0],
    [BERG, 107.6, RANE, 141.95],
    [GABE, 130.1, KITW, 114.25]
])

# Week 3
app.db.addGamesForYearAndWeek(2022, 3, [
    [CARR, 126.55, BERG, 119.75],
    [GRIG, 104.3, DVIS, 113.35],
    [SHAR, 116.3, PETE, 97.1],
    [BRKS, 96.55, FLAN, 117.8],
    [ROBL, 135.0, JOSH, 115.05],
    [ALEX, 107.95, RANE, 115.95],
    [KITW, 160.2, SHAB, 77.75],
    [PRUS, 84.5, GABE, 120.0]
])

# Week 4
app.db.addGamesForYearAndWeek(2022, 4, [
    [RANE, 142.15, CARR, 119.6],
    [SHAR, 108.85, GRIG, 139.25],
    [DVIS, 85.55, BRKS, 131.0],
    [PETE, 117.55, ROBL, 112.8],
    [FLAN, 106.95, JOSH, 110.5],
    [KITW, 158.85, ALEX, 95.3],
    [SHAB, 151.15, PRUS, 73.4],
    [BERG, 118.05, GABE, 99.2]
])

# Week 5
app.db.addGamesForYearAndWeek(2022, 5, [
    [CARR, 122.85, ALEX, 104.22],
    [GRIG, 146.4, BRKS, 142.1],
    [ROBL, 100.15, SHAR, 100.4],
    [JOSH, 90.8, DVIS, 103.8],
    [FLAN, 109.3, PETE, 100.4],
    [PRUS, 66.8, KITW, 166.8],
    [GABE, 85.9, RANE, 120.5],
    [BERG, 119.4, SHAB, 143.85]
])

# Week 6
app.db.addGamesForYearAndWeek(2022, 6, [
    [CARR, 120.15, GABE, 122.0],
    [ROBL, 140.35, GRIG, 109.65],
    [BRKS, 100.7, JOSH, 144.15],
    [SHAR, 80.9, FLAN, 78.7],
    [DVIS, 63.6, PETE, 73.6],
    [PRUS, 94.75, ALEX, 104.15],
    [KITW, 115.15, BERG, 114.85],
    [RANE, 120.4, SHAB, 57.85]
])

# Week 7
app.db.addGamesForYearAndWeek(2022, 7, [
    [SHAB, 132.0, CARR, 131.75],
    [GRIG, 105.25, JOSH, 135.9],
    [FLAN, 82.65, ROBL, 80.3],
    [PETE, 98.3, BRKS, 104.95],
    [DVIS, 67.6, SHAR, 96.5],
    [ALEX, 121.85, GABE, 99.6],
    [BERG, 100.05, PRUS, 123.35],
    [RANE, 136.35, KITW, 106.7]
])

# Week 8
app.db.addGamesForYearAndWeek(2022, 8, [
    [BRKS, 133.85, CARR, 115.45],
    [GRIG, 115.4, ALEX, 98.95],
    [FLAN, 85.55, BERG, 136.9],
    [PETE, 94.1, SHAB, 174.05],
    [DVIS, 119.15, RANE, 100.8],
    [SHAR, 137.5, KITW, 123.0],
    [ROBL, 84.35, PRUS, 164.35],
    [JOSH, 115.8, GABE, 115.3]
])

# Week 9
app.db.addGamesForYearAndWeek(2022, 9, [
    [CARR, 95.05, ROBL, 151.45],
    [ALEX, 122.15, FLAN, 113.1],
    [BERG, 119.45, PETE, 89.95],
    [SHAB, 107.1, DVIS, 118.25],
    [RANE, 111.5, SHAR, 89.05],
    [KITW, 110.55, BRKS, 94.85],
    [PRUS, 112.4, JOSH, 91.85],
    [GABE, 71.85, GRIG, 86.65]
])

# Week 10
app.db.addGamesForYearAndWeek(2022, 10, [
    [JOSH, 91.95, CARR, 143.0],
    [PETE, 139.8, ALEX, 92.1],
    [DVIS, 98.2, BERG, 130.65],
    [SHAR, 73.35, SHAB, 140.75],
    [BRKS, 131.6, RANE, 127.0],
    [ROBL, 76.55, KITW, 110.4],
    [GRIG, 88.65, PRUS, 100.35],
    [FLAN, 101.35, GABE, 78.0]
])

# Week 11
app.db.addGamesForYearAndWeek(2022, 11, [
    [CARR, 122.7, GRIG, 90.05],
    [ALEX, 92.35, DVIS, 83.65],
    [BERG, 119.5, SHAR, 132.65],
    [SHAB, 120.15, BRKS, 158.9],
    [RANE, 95.05, ROBL, 112.9],
    [KITW, 96.65, JOSH, 88.65],
    [PRUS, 92.05, FLAN, 142.15],
    [GABE, 118.45, PETE, 104.15]
])

# Week 12
app.db.addGamesForYearAndWeek(2022, 12, [
    [FLAN, 94.4, CARR, 129.15],
    [SHAR, 103.25, ALEX, 121.8],
    [BRKS, 116.4, BERG, 113.25],
    [ROBL, 107.45, SHAB, 165.3],
    [JOSH, 141.8, RANE, 140.75],
    [GRIG, 108.45, KITW, 124.95],
    [PETE, 122.6, PRUS, 83.2],
    [DVIS, 91.6, GABE, 145.4]
])

# Week 13
app.db.addGamesForYearAndWeek(2022, 13, [
    [CARR, 146.1, PETE, 133.2],
    [ALEX, 104.75, BRKS, 135.05],
    [BERG, 118.5, ROBL, 126.6],
    [SHAB, 78.2, JOSH, 123.15],
    [RANE, 99.1, GRIG, 87.25],
    [KITW, 107.45, FLAN, 107.45],
    [PRUS, 114.5, DVIS, 127.9],
    [GABE, 70.55, SHAR, 77.85]
])

# Week 14
app.db.addGamesForYearAndWeek(2022, 14, [
    [DVIS, 135.0, CARR, 90.55],
    [ROBL, 137.15, ALEX, 100.85],
    [JOSH, 140.3, BERG, 90.5],
    [GRIG, 77.85, SHAB, 99.55],
    [FLAN, 137.4, RANE, 116.05],
    [PETE, 99.65, KITW, 107.15],
    [SHAR, 42.95, PRUS, 119.15],
    [BRKS, 86.85, GABE, 99.6]
])

In [40]:
# Add Playoff Games for ____

# Round 1
app.db.addPlayoffGamesForYearAndRound(2022, 1, [
    [KITW, 1, 144.1, ROBL, 8, 109.65],
    [SHAR, 5, 111.05, SHAB, 4, 103.0],
    [FLAN, 6, 110.6, RANE, 3, 116.8],
    [JOSH, 7, 132.5, BRKS, 2, 83.05]
])

# Round 2
app.db.addPlayoffGamesForYearAndRound(2022, 2, [
    [SHAR, 5, 79.8, KITW, 1, 108.7],
    [JOSH, 7, 120.15, RANE, 3, 119.65]
])

# Round 3
app.db.addPlayoffGamesForYearAndRound(2022, 3, [
    [JOSH, 7, 71.05, KITW, 1, 63.95]
])

In [41]:
app.db.addRecordsFromGames()

### Add Active Years

In [42]:
app.db.addActiveYears()

### Add Top Three Finishes

In [43]:
app.db.top_three_finish_df = TopThreeFinishHelper().createTopThreeFinishesDF()

app.db.addTopThreeFinishesForYear(2022, [
    [JOSH, 1],
    [KITW, 2],
    [SHAR, 3]
])

## Run

---

Each of the following sections will display some table, chart, or graph to visualize NJFL data. Please see the descriptions for each section and run individual cells to see the output.

### League Managers (2011 - 2022)

Shows a table with all the managers who have participated in the NJFL since it's inception.

In [44]:
app.displayLeagueManagers2()

Name,2022
Alex Nilsson,🟩🟩🟩
Brendan Carr,🟩🟩🟩
Brooks Langley,🟩🟩🟩
Cameron Sharer,🟩🟩🟩
Davis Hodge,🟩🟩🟩
Drew Griggs,🟩🟩🟩
Gabe Sugerman,🟩🟩🟩
Jared Birnberg,🟩🟩🟩
Josh Parker,🟩🟩🟩
Kit Williams,🟩🟩🟩


### Manager Records (2011 - 2022)

Shows a table of all league managers records since the inception of the NJFL. Sort values using one of the following values:
- `sort_by.name`, `sort_by.wins`, `sort_by.losses`, `sort_by.ties`, `sort_by.win_pct`

In [45]:
app.displayManagerRecords(sort_by.win_pct)

Name,Wins,Losses,Ties,Win Pct
Kit Williams,10,3,0,76.923
Brooks Langley,9,5,0,64.286
Rhane Jones,9,5,0,64.286
Cameron Sharer,8,6,0,57.143
Utshab Chakraborty,8,6,0,57.143
Will Flanagan,7,6,0,53.846
Alex Nilsson,7,7,0,50.000
Davis Hodge,7,7,0,50.000
Josh Parker,7,7,0,50.000
Rob Lewis,7,7,0,50.000


### Manager Record by Year (2011-2022)

Shows a table of a single managers records for every year they participated in the NJFL.

Use manager name from records table above.

In [46]:
manager = "Kit Williams"

app.displayRecordsForManager(manager)

Name,Year,Wins,Losses,Ties,PF,PA
Kit Williams,2022,10,3,0,1723.85,1427.90


### Total Championships (2011-2021)

Shows a table of total championships for league managers since the inception of the NJFL.

In [47]:
app.displayTotalChampionshipsByManager()

Name,Championships
Josh Parker,1


### Top Three Finishes (2011-2021)

Shows a table of top three finishes for league managers since the inception of the NJFL.

In [48]:
app.displayTopThreeFinishes()

Name,2022,Total
Cameron Sharer,🥉,1
Josh Parker,🥇,1
Kit Williams,🥈,1


### Last Place Finishes (2022)

Shows a table of league last place finishes since this application was created.

In [ ]:
# TODO - use games table to calculate regular season record and PF/PA to determine last place finisher

### League Manager PF & PA (2022)

Shows a table of league managers all-time "points for" and "points against" since this application was created.

In [ ]:
# TODO - use games table to calculate all-time PF and PA, display total PA/PF, and average PF/PA

### League Manager PF & PA by Year

In [ ]:
# TODO - modify above function to handle parameter for year

### Average Game Point Differential

In [ ]:
# TODO

### Average PA & PF

In [ ]:
# TODO

### Highest/Lowest Points

In [ ]:
# TODO